# Roteiro de viagem com LangChain
___

Este notebook tem como objetivo entender passo a passo como construir um sistema com o LangChain.
Criar um assistente que, dado um interesse por um tipo de destino -- `praias` ou `ecoturismo`, ou outro qualquer,
crie um roteiro de viagem.

1. Cidade recomendada + motivo da recomendação
2. Restaurantes na cidade
3. Passeios culturais

___

### Parte 1: Setup e Imports

- Carregando as bibliotecas e OpenAI key

In [1]:
import os
import json

from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field

In [2]:
load_dotenv(find_dotenv())

openai_api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(
                    model="gpt-4.1",
                    temperature=0,
                    api_key=openai_api_key
                )

### Parte 2: Testar apenas o modelo com um prompt simples
___

In [3]:
response = model.invoke("Qual a cidade mais visitada no Brasil para praticantes do ecoturismo?")
print(response.content)

A cidade mais visitada do Brasil por praticantes de **ecoturismo** é **Bonito**, localizada no estado de Mato Grosso do Sul. Bonito é reconhecida nacional e internacionalmente por suas águas cristalinas, grutas, cachoeiras, trilhas e uma grande variedade de atividades de turismo sustentável, como flutuação em rios, mergulho, rapel, observação de aves e exploração de cavernas.

Além de Bonito, outros destinos importantes para o ecoturismo no Brasil incluem:

- **Foz do Iguaçu (PR)** – famosa pelas Cataratas do Iguaçu e o Parque Nacional do Iguaçu.
- **Chapada Diamantina (BA)** – conhecida por suas trilhas, cachoeiras e grutas.
- **Chapada dos Veadeiros (GO)** – famosa por suas paisagens de cerrado, cachoeiras e trilhas.
- **Amazônia (AM/PA)** – para experiências de floresta tropical, rios e biodiversidade.

No entanto, **Bonito** é frequentemente citada como o principal destino brasileiro para ecoturismo devido à infraestrutura, variedade de atrações naturais e políticas de preservação 

### Parte3: Criar Prompt Estruturado com Pydantic + JsonOutputParser
___

"* `{interesse}`: recomendação de cidade + motivo da recomendação"

In [4]:
class Destino(BaseModel):
    cidade: str = Field(description="Nome da cidade recomendada para o tipo de interesse do usuário.")
    motivo: str = Field(description="Motivo da recomendação da cidade para o interesse específicado pelo usuário.")

parser_destino = JsonOutputParser(pydantic_object=Destino)

prompt_destino = ChatPromptTemplate.from_template(
    """Sugira uma cidade com base no interesse do usuário, {interesse}.
    Explique o motivo da sugestão da cidade.\n{format_instructions}""",
    partial_variables={"format_instructions": parser_destino.get_format_instructions()}
)

chain_destino = prompt_destino | model | parser_destino

response_destino = chain_destino.invoke({"interesse": "ecoturismo"}) # objeto pydantic

In [5]:
print(response_destino)
print(type(response_destino))

{'cidade': 'Bonito', 'motivo': 'Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece rios de águas cristalinas, grutas, cachoeiras e uma grande variedade de atividades de contato com a natureza, como flutuação, trilhas e observação de fauna e flora, sendo ideal para quem busca experiências sustentáveis e preservação ambiental.'}
<class 'dict'>


In [6]:
cidade_recomendada = response_destino["cidade"]
motivo_recomendacao = response_destino["motivo"]

print(f"Cidade recomendada: {cidade_recomendada}\n")
print(f"Motivo da recomendação: {motivo_recomendacao}")

Cidade recomendada: Bonito

Motivo da recomendação: Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece rios de águas cristalinas, grutas, cachoeiras e uma grande variedade de atividades de contato com a natureza, como flutuação, trilhas e observação de fauna e flora, sendo ideal para quem busca experiências sustentáveis e preservação ambiental.


#### Parte 3.1 - Saída e análise da primeira chain:

Sugestão de cidade por `{interesse}` e `motivo da recomendação`

In [7]:
# Supondo que seu código anterior já foi executado:
# model = ...
# parser_destino = ...
# prompt_destino = ...
# chain_destino = prompt_destino | model | parser_destino
# response = chain_destino.invoke({"interesse": "ecoturismo"})

# 1. Inspecionando o tipo e acessando os atributos
print(f"O tipo da variável 'response' é: {type(response_destino)}")
print("-" * 40)
print(f"Cidade Sugerida: {cidade_recomendada}")
print(f"Motivo da Sugestão: {motivo_recomendacao}")
print("-" * 40)


# 2. Melhorando a visualização com .model_dump() e json.dumps()
print("\n✅ Saída formatada como JSON (ideal para logs e debug):")

# Usa json.dumps para formatar o dicionário como uma string JSON "bonita"
json_output = json.dumps(
    response_destino,
    indent=2,          # Adiciona indentação para legibilidade
    ensure_ascii=False # Garante que caracteres como "ç" e "ã" sejam exibidos corretamente
)

print(json_output)


O tipo da variável 'response' é: <class 'dict'>
----------------------------------------
Cidade Sugerida: Bonito
Motivo da Sugestão: Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece rios de águas cristalinas, grutas, cachoeiras e uma grande variedade de atividades de contato com a natureza, como flutuação, trilhas e observação de fauna e flora, sendo ideal para quem busca experiências sustentáveis e preservação ambiental.
----------------------------------------

✅ Saída formatada como JSON (ideal para logs e debug):
{
  "cidade": "Bonito",
  "motivo": "Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece rios de águas cristalinas, grutas, cachoeiras e uma grande variedade de atividades de contato com a natureza, como flutuação, trilhas e observação de fauna e flora, sendo ideal para quem busca experiências sustentávei

### Parte 4 - Encadear a chain de restaurante
___ 

Dado a saída da chain anterior, que foi a cidade baseada no interesse do usuário, agora é escolher restaurantes de boa qualidade caseiros e um pouco mais sofisticados.

In [8]:
class Restaurante(BaseModel):
    nome: str = Field(description="Nome do restaurante.")
    tipo: str = Field(description="Tipo de culinária do restaurante.")
    nota: float = Field(description="Nota do restaurante.")
    preco: float = Field(description="Preço médio do prato para duas pessoas.")
    descricao: str = Field(description="Descrição do restaurante.")


class ListaRestaurantes(BaseModel):
    restaurantes: list[Restaurante] = Field(description="Lista de restaurantes sugeridos")

parser_restaurante = PydanticOutputParser(pydantic_object=ListaRestaurantes)

prompt_restaurante = ChatPromptTemplate.from_template(
    """Você é um assistente especialista em gastronomia e trabalha para a agência
    de viagens.

Para a cidade {cidade}, sugira 3 restaurantes de boa qualidade caseiros e 3 mais sofisticados.

Siga estritamente as instruções de formatação abaixo:
{format_instructions}
""",
    #* formatar a saída
    partial_variables={"format_instructions": parser_restaurante.get_format_instructions()}
)


chain_restaurante = prompt_restaurante | model | parser_restaurante
cidade = cidade_recomendada

response_restaurante = chain_restaurante.invoke({
    "cidade": cidade,
    "format_instructions": parser_restaurante.get_format_instructions()
})

#*A saída será um objeto ListaRestaurantes(Pydantic)
print(response_restaurante)

#* Podemos acessa a lista facilmente
for restaurante in response_restaurante.restaurantes:
    print(f"--- \nNome: {restaurante.nome}\nTipo: {restaurante.tipo}\nNota: {restaurante.nota}")

restaurantes=[Restaurante(nome='Casa do João', tipo='Culinária regional caseira', nota=4.7, preco=120.0, descricao='Restaurante tradicional de Bonito, famoso pelo ambiente acolhedor e pratos típicos da região, como o pintado à urucum e a traíra sem espinha. Ideal para quem busca comida caseira de alta qualidade.'), Restaurante(nome='Restaurante da Zézinha', tipo='Culinária caseira sul-mato-grossense', nota=4.5, preco=90.0, descricao='Ambiente simples e familiar, com buffet variado de pratos regionais, incluindo carne de sol, arroz carreteiro e saladas frescas. Muito procurado por moradores e turistas que buscam comida saborosa e autêntica.'), Restaurante(nome='Cantinho do Peixe', tipo='Culinária caseira com foco em peixes', nota=4.6, preco=100.0, descricao='Especializado em peixes de água doce preparados de forma caseira, como pacu e pintado. Ambiente descontraído e atendimento atencioso, perfeito para famílias e grupos.'), Restaurante(nome='Juanita Restaurante', tipo='Culinária region

### Parte 5 - Encadear com passeios culturais - (texto livre)
___

In [9]:
prompt_passeios = ChatPromptTemplate.from_template(
    "Sugira 3 passeios culturais imperdíveis na cidade de {cidade}, com descrição de cada um."
)

chain_passeios = prompt_passeios | model | StrOutputParser()
cidade = cidade_recomendada

chain_passeios.invoke({"cidade": cidade})


'Claro! Embora Bonito, no Mato Grosso do Sul, seja mais famosa por suas belezas naturais e ecoturismo, a cidade também oferece experiências culturais interessantes. Aqui estão três passeios culturais imperdíveis em Bonito:\n\n---\n\n**1. Projeto Jiboia**  \n**Descrição:**  \nO Projeto Jiboia é uma iniciativa de educação ambiental que visa desmistificar o medo das serpentes e promover a conservação desses animais. Durante a visita, os participantes assistem a uma palestra interativa e divertida sobre a importância das cobras para o ecossistema, aprendem a diferenciar espécies venenosas e não venenosas e, ao final, têm a oportunidade de manusear uma jiboia de verdade. É uma experiência educativa e única, ideal para todas as idades.\n\n---\n\n**2. Casa da Memória Raída**  \n**Descrição:**  \nA Casa da Memória Raída é um espaço cultural dedicado à preservação da história e das tradições de Bonito e região. O local reúne objetos antigos, fotografias, documentos e relatos que contam a trajet

### Parte 6 - Montar a versão encadeada passo a passo:
___

Agora que temos as três partes do nosso roteiro funcionando de forma independente, vamos uni-las em uma única "super chain".

O fluxo será:
1.  **Entrada**: Receber o `{interesse}` do usuário.
2.  **Passo 1**: Executar a `chain_destino` para obter a cidade e o motivo.
3.  **Passo 2**: Usar a cidade gerada no Passo 1 para executar, em paralelo, a `chain_restaurante` e a `chain_passeios`.
4.  **Saída**: Juntar todos os resultados em um único objeto.

Para isso, usaremos `RunnablePassthrough.assign` para carregar o resultado da primeira chain e, em seguida, alimentar as outras duas.

In [10]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

#* elembrando nossas chains:
#* chain_destino: Recebe {"interesse": str} -> Retorna {"cidade": str, "motivo": str}
#* chain_restaurante: Recebe {"cidade": str} -> Retorna ListaRestaurantes(Pydantic)
#* chain_passeios: Recebe {"cidade": str} -> Retorna str

#!Vamos construir a chain principal (roteiro_chain) passo a passo

# Com a chain_restaurante corrigida, nosso encadeamento final fica mais limpo.
roteiro_chain = (
    RunnablePassthrough.assign(
        destino_info=chain_destino
    )
    |
    RunnablePassthrough.assign(
        sugestoes=RunnableParallel(
            restaurantes=(
                # A lambda agora só precisa passar a cidade, como esperado!
                (lambda x: {"cidade": x['destino_info']['cidade']})
                | chain_restaurante
            ),
            passeios=(
                (lambda x: {"cidade": x['destino_info']['cidade']})
                | chain_passeios
            )
        )
    )
)

print("✅ Roteiro Chain (versão corrigida) montada com sucesso!")
print("Tipo do objeto:", type(roteiro_chain))

✅ Roteiro Chain (versão corrigida) montada com sucesso!
Tipo do objeto: <class 'langchain_core.runnables.base.RunnableSequence'>


### Parte 7 - Executando a Chain Completa e Apresentando o Roteiro
___

Com nossa `roteiro_chain` montada, basta chamá-la uma única vez com o interesse do usuário. O resultado será um dicionário aninhado com todas as informações que pedimos.

Vamos invocar a chain e depois formatar a saída para apresentar um roteiro de viagem claro e organizado.

In [11]:
#*. Definir o interesse do usuário
interesse_usuario = "ecoturismo"

# 2. Invocar a chain completa
print(f"Gerando roteiro de viagem para o interesse: '{interesse_usuario}'...")
roteiro_final = roteiro_chain.invoke({"interesse": interesse_usuario})
print("Roteiro gerado! Formatando a saída...")

# 3. Apresentar o resultado de forma organizada

# Extraindo as informações do dicionário de resultado
cidade = roteiro_final['destino_info']['cidade']
motivo = roteiro_final['destino_info']['motivo']
lista_de_restaurantes = roteiro_final['sugestoes']['restaurantes'].restaurantes
sugestao_de_passeios = roteiro_final['sugestoes']['passeios']

# Imprimindo o roteiro formatado
print("\\n" + "="*50)
print(f"🚌 SEU ROTEIRO DE VIAGEM PARA {cidade.upper()} 🚌")
print("="*50 + "\\n")

print(f"📍 Destino Sugerido: {cidade}")
print(f"💡 Motivo: {motivo}\\n")
print("-" * 50 + "\\n")

print("🍽️ Sugestões de Restaurantes:\\n")
for r in lista_de_restaurantes:
    print(f"  - Nome: {r.nome} ({r.tipo})")
    print(f"    Descrição: {r.descricao}")
    print(f"    Nota: {r.nota} | Preço Médio (2p): R$ {r.preco:.2f}\\n")
print("-" * 50 + "\\n")

print("🏞️ Sugestões de Passeios Culturais:\\n")
print(sugestao_de_passeios)
print("\\n" + "="*50)

Gerando roteiro de viagem para o interesse: 'ecoturismo'...
Roteiro gerado! Formatando a saída...
\n==================================================
🚌 SEU ROTEIRO DE VIAGEM PARA BONITO 🚌
==================================================\n
📍 Destino Sugerido: Bonito
💡 Motivo: Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece rios de águas cristalinas, grutas, cachoeiras e uma grande variedade de atividades de contato com a natureza, como flutuação, trilhas e observação de fauna e flora, sendo ideal para quem busca experiências sustentáveis e preservação ambiental.\n
--------------------------------------------------\n
🍽️ Sugestões de Restaurantes:\n
  - Nome: Casa do João (Culinária regional caseira)
    Descrição: Restaurante tradicional de Bonito, famoso pelo ambiente acolhedor e pratos típicos sul-mato-grossenses preparados de forma caseira, como o pintado à urucum e a traíra sem espinha.
    Not